# Загрузка и обработка файлов

## Скачивание данных, распаковка и запись

In [1]:
'''
import os
from urllib.request import urlretrieve
import zipfile

# Создание папки data, если не существует
os.makedirs('./data', exist_ok=True)

# Пути к файлам
train_zip_path = './data/train.zip'
valid_zip_path = './data/valid.zip'

# Скачивание архивов
urlretrieve('https://www.dropbox.com/scl/fi/bel6gt6vsb3onahlxvyjc/train_fix.zip?rlkey=q2wscp6wv9j2hbk07y1mbcm54&dl=1', train_zip_path)
urlretrieve('https://www.dropbox.com/scl/fi/cwwblwhvqgwubb8a4xg90/valid.zip?rlkey=mow899lvyawq4wku2m8lfvrh3&dl=1', valid_zip_path)

# Распаковка
with zipfile.ZipFile(train_zip_path, 'r') as zip_ref:
    zip_ref.extractall('./data/train')

with zipfile.ZipFile(valid_zip_path, 'r') as zip_ref:
    zip_ref.extractall('./data/valid')

print("Готово! Архивы скачаны и распакованы в папку ./data")
'''


'\nimport os\nfrom urllib.request import urlretrieve\nimport zipfile\n\n# Создание папки data, если не существует\nos.makedirs(\'./data\', exist_ok=True)\n\n# Пути к файлам\ntrain_zip_path = \'./data/train.zip\'\nvalid_zip_path = \'./data/valid.zip\'\n\n# Скачивание архивов\nurlretrieve(\'https://www.dropbox.com/scl/fi/bel6gt6vsb3onahlxvyjc/train_fix.zip?rlkey=q2wscp6wv9j2hbk07y1mbcm54&dl=1\', train_zip_path)\nurlretrieve(\'https://www.dropbox.com/scl/fi/cwwblwhvqgwubb8a4xg90/valid.zip?rlkey=mow899lvyawq4wku2m8lfvrh3&dl=1\', valid_zip_path)\n\n# Распаковка\nwith zipfile.ZipFile(train_zip_path, \'r\') as zip_ref:\n    zip_ref.extractall(\'./data/train\')\n\nwith zipfile.ZipFile(valid_zip_path, \'r\') as zip_ref:\n    zip_ref.extractall(\'./data/valid\')\n\nprint("Готово! Архивы скачаны и распакованы в папку ./data")\n'

## Разделим виды бабочек по классам в разные папки

In [2]:
'''
import os
import shutil

def restructure_dataset(source_dir):
    for file_name in os.listdir(source_dir):
        if not file_name.endswith(('.jpg', '.jpeg', '.png')):
            continue

        # Извлекаем имя класса из имени файла (до первой скобки)
        class_name = file_name.split(' (')[0]
        class_dir = os.path.join(source_dir, class_name)

        # Создаем папку, если нужно
        os.makedirs(class_dir, exist_ok=True)

        # Перемещаем файл в папку
        src_path = os.path.join(source_dir, file_name)
        dst_path = os.path.join(class_dir, file_name)
        shutil.move(src_path, dst_path)

# Применяем к train и valid
restructure_dataset('./data/train')
restructure_dataset('./data/valid')

print("Структура папок исправлена")
'''

'\nimport os\nimport shutil\n\ndef restructure_dataset(source_dir):\n    for file_name in os.listdir(source_dir):\n        if not file_name.endswith((\'.jpg\', \'.jpeg\', \'.png\')):\n            continue\n\n        # Извлекаем имя класса из имени файла (до первой скобки)\n        class_name = file_name.split(\' (\')[0]\n        class_dir = os.path.join(source_dir, class_name)\n\n        # Создаем папку, если нужно\n        os.makedirs(class_dir, exist_ok=True)\n\n        # Перемещаем файл в папку\n        src_path = os.path.join(source_dir, file_name)\n        dst_path = os.path.join(class_dir, file_name)\n        shutil.move(src_path, dst_path)\n\n# Применяем к train и valid\nrestructure_dataset(\'./data/train\')\nrestructure_dataset(\'./data/valid\')\n\nprint("Структура папок исправлена")\n'

In [3]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn as nn
import torch.nn.functional as F

In [4]:
import os

max_threads = int(os.cpu_count() * 0.8)
os.environ["OMP_NUM_THREADS"] = str(max_threads)
os.environ["MKL_NUM_THREADS"] = str(max_threads)
torch.set_num_threads(max_threads)

# Подготовка данных

In [5]:
# Аугментации и нормализация
transform_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

transform_valid = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
])

In [6]:
# Загрузка датасета
train_data_full = datasets.ImageFolder(root='./data/train', transform=transform_train)
valid_data = datasets.ImageFolder(root='./data/valid', transform=transform_valid)

# Разделение тренировочной на обучающую и внутреннюю валидацию
train_size = int(0.8 * len(train_data_full))
val_size = len(train_data_full) - train_size
train_data, train_val_data = random_split(train_data_full, [train_size, val_size])

In [7]:
# DataLoader'ы
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = DataLoader(train_val_data, batch_size=32)
valid_loader = DataLoader(valid_data, batch_size=32)

In [8]:
# Число классов
num_classes = len(train_data_full.classes)
print(f"Всего классов: {num_classes}")

Всего классов: 75


# Работа с моделью

## Создание модели

In [9]:
class ButterflyCNN(nn.Module):
    def __init__(self, num_classes):
        super(ButterflyCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        
        self.conv3x3 = nn.Conv2d(64, 32, kernel_size=3, padding=1)
        self.conv5x5 = nn.Conv2d(64, 32, kernel_size=5, padding=2)

        self.conv_merge = nn.Conv2d(64, 256, kernel_size=3, padding=1)

        self.pool = nn.MaxPool2d(2, 2)

        self.fc1 = nn.Linear(256 * 14 * 14, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))     # -> [B, 16, 112, 112]
        x = self.pool(F.relu(self.conv2(x)))     # -> [B, 32, 56, 56]
        x = self.pool(F.relu(self.conv3(x)))     # -> [B, 64, 28, 28]

        # Параллельные свёртки на 64 каналах
        x3 = F.relu(self.conv3x3(x))             # -> [B, 32, 28, 28]
        x5 = F.relu(self.conv5x5(x))             # -> [B, 32, 28, 28]

        # Объединение по каналам
        x = torch.cat([x3, x5], dim=1)           # -> [B, 64, 28, 28]

        # Свертка до 256 каналов
        x = self.pool(F.relu(self.conv_merge(x))) # -> [B, 256, 14, 14]

        x = x.view(-1, 256 * 14 * 14)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


## Обучение модели

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Обучение начнётся на устройстве: {device}")

model = ButterflyCNN(num_classes=num_classes).to(device)
best_model_path = './model/best_model_.pth'
if os.path.exists(best_model_path):
    model.load_state_dict(torch.load(best_model_path, map_location=device))
    print("Загружены старые веса из", best_model_path)
else:
    print("Весов предыдущей модели не найдено, начнётся обучение с нуля")

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Параметры обучения
epochs = 20
best_val_acc = 0.0

try:
    for epoch in range(1, epochs + 1):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()

        train_loss = running_loss / total
        train_acc = 100 * correct / total
        print(f"[Epoch {epoch}/{epochs}] "
            f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")

        torch.save(model.state_dict(), './model/best_model_.pth')
        print("Model improved and saved.")
except KeyboardInterrupt:
    print("Обучение приостановлено")
except Exception as e:
    print("Произошла ошибка:", e)

Обучение начнётся на устройстве: cpu


C:\Users\Alexey\AppData\Local\Temp\ipykernel_14876\998731076.py:7: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path, map_locati

Загружены старые веса из ./model/best_model_.pth
Обучение приостановлено


In [ ]:
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)
            _, preds = outputs.max(1)
            val_total += labels.size(0)
            val_correct += (preds == labels).sum().item()

    val_loss /= val_total
    val_acc = 100 * val_correct / val_total
    print(f"[Epoch {epoch}/{epochs}] "
          f" Val Loss: {val_loss:.4f},  Val Acc: {val_acc:.2f}%")

[Epoch 20/20]  Val Loss: 1.6916,  Val Acc: 53.42%
